In [1]:
import requests
import json
import time
import math
import re
import calendar
import dateutil.parser as parser
from dateutil.relativedelta import relativedelta
from datetime import datetime, timezone
import yaml
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
from sklearn.neighbors import NearestNeighbors
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer
import os, sys, glob
import kaleido
from PIL import Image
from fpdf import FPDF
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [2]:
def convert_date(utc_time): 
    parsed_date = parser.parse(utc_time)
    var_date=parsed_date.date()
    var_time=parsed_date.time()
    var_f_time=var_time.hour
    var_julian_date=parsed_date.timetuple().tm_yday
    var_weekday=parsed_date.weekday()
    var_weekday_name=calendar.day_name[parsed_date.weekday()]
    return var_date, var_time, var_f_time, var_julian_date, var_weekday, var_weekday_name

In [3]:
def viz(data):
    fig = go.Figure(data=[go.Candlestick(x=data['UTC_Time'],
                open=data['Open'],
                high=data['High'],
                low=data['Low'],
                close=data['Close'])])

    fig.update_layout(xaxis_rangeslider_visible=False,
                      title_font_color="blue",
                      title_font_size = 20)
    
    fig.update_xaxes(rangebreaks=[dict(bounds=["sat", "mon"])])   
    fig.show()

In [4]:
def find_k_similar_candles(candle_id, dataset, k=4):
    indices=[]
    distances = []
    output = []
    model_knn = NearestNeighbors(metric = 'euclidean', algorithm = 'auto') 
    model_knn.fit(dataset)
    
    #metric = 'euclidean' or 'cosine' or 'manhattan' or 'mahalanobis'
    
    distances, indices = model_knn.kneighbors(dataset.iloc[candle_id,:].values.reshape(1,-1),
                                              n_neighbors = k)

    for i in range(0,len(distances.flatten())):
        if i!=0:
            
            output.append ([dataset.index[indices.flatten()[i]],
                            distances.flatten()[i],
#                           dataset.iloc[indices.flatten()[i]]['O-H'],dataset.iloc[indices.flatten()[i]]['O-L'],dataset.iloc[indices.flatten()[i]]['O-C'],dataset.iloc[indices.flatten()[i]]['H-L'],dataset.iloc[indices.flatten()[i]]['H-C'],dataset.iloc[indices.flatten()[i]]['L-C'],
                           dataset.iloc[indices.flatten()[i]]['col_1'],dataset.iloc[indices.flatten()[i]]['col_2'],dataset.iloc[indices.flatten()[i]]['col_3'],
#                           dataset.iloc[indices.flatten()[i]]['col_4'],dataset.iloc[indices.flatten()[i]]['col_5'],
                           dataset.iloc[indices.flatten()[i]]['F_SMA_10'],dataset.iloc[indices.flatten()[i]]['F_SMA_20'],
                           ])
    
    output = pd.DataFrame(output)
    output.columns = ['Indice','Distance',
#                      'O-H','O-L','O-C','H-L','H-C','L-C',
                      'col_1','col_2','col_3',
#                      'col_4','col_5',
                      'F_SMA_10','F_SMA_20'
                     ]
   # display (output)
    
    return indices, distances

# <font color='red'>Test Configs</font>

In [5]:
Test_Candle = [2108, 8371, 4614, 8207, 5172, 3342, 121, 8607, 6387, 9303, 3835, 2134, 6930, 6858, 7301, 8099, 985, 5419, 7471, 1457, 8910, 3841, 704, 7308, 8734, 7172, 7385, 1613, 5421, 7061, 1088, 4383, 9123, 166, 9414, 3899, 4915, 988, 5856, 4308, 742, 5213, 6201, 3898, 1346, 9255, 2498, 6305, 4755, 837, 4628, 805, 3896, 5168, 237, 6147, 4220, 3683, 6668, 989, 216, 4455, 3231, 3456, 8141, 629, 8250, 8351, 8648, 2499, 1818, 1253, 22, 3337, 8189, 2423, 5209, 1426, 4332, 3663, 8871, 1620, 8493, 7271, 214, 6443, 97, 2524, 1767, 7839, 3644, 6230, 7605, 4921, 5945, 2162, 3773, 7216, 2936]
#Test_Candle = [2108, 8371, 4614, 8207]

In [6]:
filename = 'BackTest_EUR_USD.csv'
data = pd.read_csv(filename)

In [7]:
data.shape

(9646, 29)

In [8]:
data.head(2)

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,...,Direction,col_1,col_2,col_3,col_4,col_5,SMA_10,SMA_20,F_SMA_10,F_SMA_20
0,2015-08-18,21:00:00,21,230,1,Tuesday,2015-08-18T21:00:00.000000000Z,967,1.10242,1.10391,...,1,-0.00138,0.00011,0.00035,0.00052,0.00004,1.104040,1.105959,-0.000240,-0.002159
1,2015-08-19,01:00:00,1,231,2,Wednesday,2015-08-19T01:00:00.000000000Z,1665,1.10380,1.10526,...,1,-0.00109,0.00037,0.00019,-0.00138,0.00052,1.103502,1.105358,0.001388,-0.000468


# <font color='red'>CANDLE LOOP</font>

In [9]:
result_output = pd.DataFrame({'Candle_No':[],
                              'Current_Market_Fit':[],
                              'Current_Market':[],
                              'Current_Market_Stoploss':[],
                              'Rec1':[],
                              'Rec1_P':[],
                              'Rec2':[],
                              'Rec2_P':[],
                              'Rec3':[],
                              'Rec3_P':[]
                             })

for candle_no in Test_Candle:
    current_market = "currnet_market.csv"
    data = pd.read_csv(filename)
    data = data.iloc[candle_no:candle_no+10]
    data.to_csv(current_market, header = True, index = False)
    data = pd.read_csv(current_market)

    data['candleno'] = range (1, len(data) + 1)
    X = data['candleno'].values.reshape(-1, 1)
    Y = data['Close'].values.reshape(-1, 1)
    linear_regressor = LinearRegression()
    linear_regressor.fit(X, Y)
    y_pred = linear_regressor.predict(X) 
    
    Current_Market_Fit = r2_score(Y, y_pred).round(2)*100 # 1.  Saved Data -> current market , fit
    #print(Current_Market_Fit)
    coeficient = (linear_regressor.coef_)

    if coeficient > 0:
        Current_Market='Bullish'    #  2. Saved Data  -> current market
        Current_Market_Stoploss= 0 #BUY_SL #  3. Saved Data  -> current market , buy stoploss
    else:
        Current_Market = 'Bearish'     # 2. Saved Data  -> current market
        Current_Market_Stoploss= 0 #SELL_SL # 3. Saved Data  -> current market , sell stoploss
    
    data = pd.read_csv(filename)
    data = data.drop(columns=['Volume','Weekday','Date','Time',
                          'Weekday_Name','UTC_Time','Direction',
                          'Open', 'High', 'Low', 'Close',
                          'O-H','O-L','O-C','H-L','H-C','L-C',
                          'SMA_10','SMA_20',
                          'f_time','julian_date','ATR_14',
#                          'col_1','col_2','col_3',
                          'col_4','col_5',
#                          'F_SMA_10','F_SMA_20'
                         ])

    indices, distances = find_k_similar_candles (candle_no,data)
    indices = indices[0:1][0]
    
    predicted_output = []
    for indice in indices[1:4]:
             
        Predicted_Market_Fit =0
        Predicted_Trade=''
    
        data = pd.read_csv(filename) 
        data = data.iloc[indice:indice+30]

        data['candleno'] = range (1, len(data) + 1)
        X = data['candleno'].values.reshape(-1, 1)
        Y = data['Close'].values.reshape(-1, 1)
        linear_regressor = LinearRegression()
        linear_regressor.fit(X, Y)
        y_pred = linear_regressor.predict(X)

        Predicted_Market_Fit= r2_score(Y, y_pred).round(2)*100 #   4,5,6 Saved Data  -> Predicted market --- Fit
        coeficient = (linear_regressor.coef_)

        if coeficient > 0:
            Predicted_Trade= 'BUY' #  4,5,6 Saved Data  -> Predicted market
        else:
            Predicted_Trade = 'SELL' #  4,5,6 Saved Data  -> Predicted market
        
        predicted_output.append([Predicted_Market_Fit,Predicted_Trade])

        
    result = {'Candle_No': candle_no,
              'Current_Market_Fit': Current_Market_Fit,
              'Current_Market': Current_Market,
              'Current_Market_Stoploss':Current_Market_Stoploss,
              'Rec1': predicted_output[0][0],
              'Rec1_P': predicted_output[0][1],
              'Rec2': predicted_output[1][0],
              'Rec2_P': predicted_output[1][1],
              'Rec3': predicted_output[2][0],
              'Rec3_P': predicted_output[2][1],
             }
    
    result_output = result_output.append(result, ignore_index = True)

In [10]:
result_output.to_csv('Back_Test_Result.csv', header = True, index = False)
result_output = pd.read_csv('Back_Test_Result.csv')

In [11]:
result_output.head(100)

,Candle_No,Current_Market_Fit,Current_Market,Current_Market_Stoploss,Rec1,Rec1_P,Rec2,Rec2_P,Rec3,Rec3_P
0,2108.0,29.0,Bearish,0.0,22.0,BUY,55.0,SELL,46.0,BUY
1,8371.0,64.0,Bearish,0.0,65.0,BUY,1.0,SELL,35.0,SELL
2,4614.0,0.0,Bullish,0.0,90.0,BUY,3.0,SELL,61.0,SELL
3,8207.0,63.0,Bullish,0.0,71.0,BUY,49.0,BUY,92.0,BUY
4,5172.0,92.0,Bullish,0.0,0.0,SELL,81.0,SELL,82.0,BUY
5,3342.0,29.0,Bearish,0.0,0.0,SELL,18.0,BUY,82.0,SELL
6,121.0,54.0,Bullish,0.0,66.0,BUY,59.0,BUY,18.0,SELL
7,8607.0,95.0,Bearish,0.0,92.0,SELL,26.0,SELL,50.0,SELL
8,6387.0,43.0,Bullish,0.0,80.0,SELL,18.0,BUY,4.0,SELL
9,9303.0,78.0,Bullish,0.0,0.0,BUY,39.0,SELL,25.0,SELL


In [12]:
result_output

,Candle_No,Current_Market_Fit,Current_Market,Current_Market_Stoploss,Rec1,Rec1_P,Rec2,Rec2_P,Rec3,Rec3_P
0,2108.0,29.0,Bearish,0.0,22.0,BUY,55.0,SELL,46.0,BUY
1,8371.0,64.0,Bearish,0.0,65.0,BUY,1.0,SELL,35.0,SELL
2,4614.0,0.0,Bullish,0.0,90.0,BUY,3.0,SELL,61.0,SELL
3,8207.0,63.0,Bullish,0.0,71.0,BUY,49.0,BUY,92.0,BUY
4,5172.0,92.0,Bullish,0.0,0.0,SELL,81.0,SELL,82.0,BUY
...,...,...,...,...,...,...,...,...,...,...
94,5945.0,49.0,Bearish,0.0,8.0,SELL,79.0,SELL,26.0,BUY
95,2162.0,70.0,Bearish,0.0,15.0,SELL,7.0,SELL,71.0,BUY
96,3773.0,99.0,Bullish,0.0,35.0,SELL,1.0,BUY,67.0,SELL
97,7216.0,8.0,Bearish,0.0,35.0,SELL,89.0,SELL,57.0,BUY
